In [1]:
import os
import cv2
import face_recognition
from sklearn.neighbors import KNeighborsClassifier
import joblib
import time

In [2]:
def load_face_encodings(directory):
    face_encodings = []
    names = []

    for name in os.listdir(directory):
        person_dir = os.path.join(directory, name)
        if not os.path.isdir(person_dir):
            continue

        for image_file in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_file)
            image = face_recognition.load_image_file(image_path)
            face_encodings_in_image = face_recognition.face_encodings(image)
            if len(face_encodings_in_image) > 0:
                face_encoding = face_encodings_in_image[0]
                face_encodings.append(face_encoding)
                names.append(name)

    return face_encodings, names

In [3]:
def recognize_faces(frame, model, known_encodings, known_names, tolerance=0.6):
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    recognized_names = []
    for face_encoding in face_encodings:
        distances = face_recognition.face_distance(known_encodings, face_encoding)
        min_distance_idx = distances.argmin()
        min_distance = distances[min_distance_idx]

        if min_distance <= tolerance:
            name = known_names[min_distance_idx]
            recognized_names.append(name)
        else:
            recognized_names.append("Unknown Person")

    return recognized_names

In [4]:
def main():
    image_directory = 'D:/Data Science/face_recognizer/training'
    model_filename = 'D:/Data Science/face_recognizer/model/face_recognition_model.joblib'
    face_detection_frequency = 144
    distance_tolerance = 0.6
    display_duration = 5

    if os.path.exists(model_filename):
        model_data = joblib.load(model_filename)
        face_encodings, names = model_data['face_encodings'], model_data['names']
    else:
        face_encodings, names = load_face_encodings(image_directory)
        model_data = {'face_encodings': face_encodings, 'names': names}
        joblib.dump(model_data, model_filename)

    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(face_encodings, names)

    video_capture = cv2.VideoCapture(0)  # Use the default backend

    # Set camera resolution to capture at 144 FPS
    video_capture.set(cv2.CAP_PROP_FPS, 144)

    frame_count = 0
    recognized_names_start_time = None

    while True:
        ret, frame = video_capture.read()

        if not ret:
            print("Failed to capture frame.")
            break

        frame_count += 1

        frame = cv2.flip(frame, 1)

        if frame_count % face_detection_frequency == 0:
            recognized_names = recognize_faces(frame, model, face_encodings, names, distance_tolerance)

            if not recognized_names:
                recognized_names = ["Unknown Person"]

            recognized_names_start_time = time.time()

        if recognized_names_start_time is not None:
            time_since_recognition = time.time() - recognized_names_start_time
            if time_since_recognition <= display_duration:
                for name in recognized_names:
                    cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            else:
                recognized_names_start_time = None

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    main()